# Covid Texting/Email
Inspired by this [blog post](https://towardsdatascience.com/how-to-track-coronavirus-with-python-a5320b778c8e)

In [223]:
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import itertools
import pandas as pd
import corona
import numpy as np

In [16]:
class Coronavirus():
  def __init__(self, link):
    self.driver = webdriver.Chrome()
    self.driver.get(link)

In [ ]:
# bot_world = Coronavirus('https://www.worldometers.info/coronavirus/')
# bot_us = Coronavirus('https://www.worldometers.info/coronavirus/country/us/')
# bot_la = Coronavirus('http://ldh.la.gov/Coronavirus/')

## Louisiana Parishes

In [42]:
bot_la = Coronavirus('https://www.arcgis.com/apps/opsdashboard/index.html#/69b726e2b82e408f89c3a54f96e8f776')


In [48]:
lines = bot_la.driver.find_elements_by_xpath('//*[@id="ember56"]/div/nav/span/div/div/p[2]/strong')
lines = [line.get_attribute("innerHTML") for line in lines]


In [49]:
# https://stackoverflow.com/questions/1624883/alternative-way-to-split-a-list-into-groups-of-n
def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [50]:
grouped_list = list(grouper(3, lines))

In [51]:
df = pd.DataFrame(grouped_list, columns =['Parish', 'Cases', 'Deaths']) 

In [73]:
df

,Parish,Cases,Deaths
0,Orleans,451,15
1,Jefferson,184,3
2,Catahoula,1,1
3,St. James,1,1
4,St. Tammany,25,0
...,...,...,...
60,Vermilion,0,0
61,Vernon,0,0
62,West Carroll,0,0
63,West Feliciana,0,0


In [59]:
parish_lens = [len(p) for p in df.Parish.values]

In [65]:
max_val = max(parish_lens)


In [72]:
df['Parish'] = ['{:26}'.format(p) for p in df.Parish.values]


In [70]:
['{:26}'.format(p) for p in df.Parish.values]


['Orleans                   ',
 'Jefferson                 ',
 'Catahoula                 ',
 'St. James                 ',
 'St. Tammany               ',
 'East Baton Rouge          ',
 'Ascension                 ',
 'Caddo                     ',
 'St. Bernard               ',
 'Lafourche                 ',
 'Terrebonne                ',
 'Parish Under Investigation',
 'St. John the Baptist      ',
 'St. Charles               ',
 'Lafayette                 ',
 'Bossier                   ',
 'Plaquemines               ',
 'Calcasieu                 ',
 'Ouachita                  ',
 'Rapides                   ',
 'St. Landry                ',
 'Tangipahoa                ',
 'West Baton Rouge          ',
 'De Soto                   ',
 'Evangeline                ',
 'Iberia                    ',
 'Iberville                 ',
 'Livingston                ',
 'Washington                ',
 'Acadia                    ',
 'Assumption                ',
 'Avoyelles                 ',
 'Beaure

## US States

In [215]:
bot_us = corona.get_us_bot()


In [66]:
df_us.to_csv('us_states_df.csv')

In [132]:
us_data = pd.read_csv('us_states_df.csv')
us_data.drop(columns="Unnamed: 0", inplace=True)
us_data.State.replace("<strong>Total:</strong>", "Total", inplace=True)

total = us_data[53:54]
# us_data[0:51]#.sort_values(by='State')


In [127]:
# this is for from the dataframe
us_data.NewCases = us_data.NewCases.str.replace("+","")
us_data.NewCases = us_data.NewCases.str.replace(",","").fillna(0).astype(int)

us_data.TotalCases = us_data.TotalCases.str.replace(",","").fillna(0).astype(int)
us_data.TotalDeaths = us_data.TotalDeaths.fillna(0).astype(int)
us_data.NewDeaths = us_data.NewDeaths.fillna(0).astype(int)

us_data["CasePercentInc"] = us_data.NewCases.str.replace(",","").fillna(0).astype(int) / (us_data.TotalCases.str.replace(",","").fillna(0).astype(int) - us_data.NewCases.str.replace(",","").fillna(0).astype(int))
us_data["DeathPercentInc"] = us_data.NewDeaths / (us_data.TotalDeaths - us_data.NewDeaths)


In [72]:
total

,State,TotalCases,NewCases,TotalDeaths,NewDeaths
53,Total,"52,447","8,713",674.0,121.0


In [60]:
# pd.set_option('display.max_rows', us_data.shape[0]+1)

In [220]:
# this is for directly from the source
df_us = corona.get_states_df(bot_us)

us_data = df_us.copy()
us_data.State.replace("<strong>Total:</strong>", "Total", inplace=True)
total = us_data[53:54]
us_data = us_data[0:51]#.sort_values(by='State')

us_data.NewCases = us_data.NewCases.str.replace("+","")
us_data.NewCases.str.replace(",","").fillna(0)
us_data.replace(r'^\s*$', np.nan, regex=True, inplace=True)
us_data["CasePercentInc"] = us_data.NewCases.str.replace(",","").fillna(0).astype(int) / (us_data.TotalCases.str.replace(",","").fillna(0).astype(int) - us_data.NewCases.str.replace(",","").fillna(0).astype(int))
us_data["DeathPercentInc"] = us_data.NewDeaths.fillna(0).astype(int) / (us_data.TotalDeaths.fillna(0).astype(int) - us_data.NewDeaths.fillna(0).astype(int))
                                                                              
   

In [182]:

"{:.1f}".format(us_data.CasePercentInc.values[0] * 100)

'22.9'

In [224]:
m = corona.get_us_state(bot_us, 'montana')

      State TotalCases NewCases TotalDeaths NewDeaths stripped_state
45  Montana         46       +1                              montana


In [211]:
m.State.dtypes

dtype('O')

In [212]:
m.NewDeaths.fillna(0)

45    
Name: NewDeaths, dtype: object

In [225]:
m

,State,TotalCases,NewCases,TotalDeaths,NewDeaths,stripped_state
45,Montana,46,+1,,,montana


In [246]:
def get_states_df(bot_us):
	states = bot_us.driver.find_elements_by_xpath("//tbody/tr/td")
	lines = [" ".join(state.get_attribute("innerHTML").split()) for state in states]
	grouped_list = list(corona.grouper(7, lines))
	df = pd.DataFrame(grouped_list).iloc[:,0:5]
	df.columns =['State', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths']
	df.State.replace("<strong>Total:</strong>", "Total", inplace=True)
	return df[0:54]

In [260]:
df = get_states_df(bot_us)


In [261]:
df['stripped_state'] = [corona.clean_text(p) for p in df.State.values]
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.NewCases = df.NewCases.str.replace("+","").fillna(0)
df.NewDeaths = df.NewDeaths.str.replace("+","").fillna(0)


In [262]:
df.NewCases.str.replace(",","").fillna(0)#.astype(int)

0     4790
1      831
2      361
3        0
4      463
5      250
6      185
7      216
8      382
9      223
10     207
11      20
12       0
13      52
14     203
15     122
16      49
17      41
18     106
19      61
20      43
21      92
22      71
23      41
24      36
25      33
26      27
27      59
28      21
29      19
30      18
31      33
32       0
33      19
34      18
35      11
36      25
37       0
38      16
39      20
40      23
41       0
42      34
43       0
44      11
45       1
46       4
47       2
48       2
49       4
50       4
51       0
52       0
53    9249
Name: NewCases, dtype: object

In [263]:
state = "montana"

In [264]:
df['stripped_state'] = [corona.clean_text(p) for p in df.State.values]
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.NewCases = df.NewCases.str.replace("+","").fillna(0)
df.NewDeaths = df.NewDeaths.str.replace("+","").fillna(0)
df.TotalDeaths = df.TotalDeaths.fillna(0)

df["CasePercentInc"] = df.NewCases.str.replace(",","").fillna(0).astype(int) / (df.TotalCases.str.replace(",","").fillna(0).astype(int) - df.NewCases.str.replace(",","").fillna(0).astype(int))
df["DeathPercentInc"] = df.NewDeaths.fillna(0).astype(int) / (df.TotalDeaths.fillna(0).astype(int) - df.NewDeaths.fillna(0).astype(int))

state = df[df.stripped_state == state]

In [265]:
state_name = state.State.values[0]
total_cases = state.TotalCases.values[0]
new_cases = state.NewCases.values[0]  or 0
case_percent = str("{:.1f}".format(state.CasePercentInc.values[0] * 100)) + "%"
total_deaths = state.TotalDeaths.values[0] or 0
new_deaths = state.NewDeaths.values[0] or 0
death_percent = str("{:.1f}".format(state.DeathPercentInc.values[0] * 100)) + "%"



In [266]:
state

,State,TotalCases,NewCases,TotalDeaths,NewDeaths,stripped_state,CasePercentInc,DeathPercentInc
45,Montana,46,1,0,0,montana,0.022222,NaN
